In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="72eb7803-e874-44cb-b6d9-33f2fa3eb88c",
    resource_group_name="vpoutotrg",
    workspace_name="mlstudio-groupe4"
)

data_asset = ml_client.data.get("allo_cine", version="2.2.1")

df = pd.read_parquet(data_asset.path)
df = df[df['french_first_week_boxoffice'].notna()]
df["date"] = pd.to_datetime(df["date"], format='%Y-%m-%d', errors='coerce')
df = df[df['date'].notna()]
df.head()

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",4.4,2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,NaN,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",3.5,2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,NaN,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html
2,[Thom Hoffman],2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,NaN,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",3.3,2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,NaN,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...


In [2]:
file_name = "cncAffluence"
version="2.0.0"

data_asset = ml_client.data.get(file_name, version=version)

df_afluence = pd.read_parquet(data_asset.path)
df_afluence.head()

,box_office,is_prediction
month,,
2009-01-01,15.148333,0
2009-02-01,19.348000,0
2009-03-01,20.087500,0
2009-04-01,16.975000,0
2009-05-01,14.528333,0


In [3]:
file_name = "cncAffluence"
version="2.0.1"
data_asset = ml_client.data.get(file_name, version=version)

df_afluence_filled = pd.read_parquet(data_asset.path)
df_afluence_filled.head()

,box_office,is_prediction
month,,
1989-01-01,9.822903,1.0
1989-02-01,11.884946,1.0
1989-03-01,11.417790,1.0
1989-04-01,10.506531,1.0
1989-05-01,9.071718,1.0


In [4]:
file_name = "actors_jpbox"
version="2.0"

data_asset = ml_client.data.get(file_name, version=version)

df_actors = pd.read_parquet(data_asset.path)
df_actors['name'] = df_actors["name"].apply(lambda x : x.replace(" ", "").replace("-", "").replace("_", "").strip().lower())
df_actors.set_index('name', inplace=True)
df_actors.head()

,amount_of_film_played_in,boxoffice_average,boxoffice_average_first_role,boxoffice_total,boxoffice_total_first_role,nationality,rank
name,,,,,,,
jasonsegel,15,197655,284469,3360134,2275749,américaine,551
katehudson,18,99462,94796,3083328,1801124,américaine,552
mirasorvino,16,81984,103371,3033394,1964043,américaine,553
alisonlohman,11,220324,211238,2643888,1056192,américaine,554
timallen,9,92895,121477,1579207,1579207,américaine,555


In [5]:
import json

target_encoding_list = {}
for col in ['genre', 'langage', 'nationality', 'directors', 'actors']:
    df[col] = df[col].apply(lambda x : x if type(x) == np.ndarray else ["no value"])
    col_res = []
    col_boxoffice_means = {}
    for col_list in df[col]:
        col_res.extend(col_list)
    unique_col = list(set(col_res))
    for item in unique_col:
        df_mask = df[df[col].apply(lambda x : item in x )]
        mean_value = df_mask['french_first_week_boxoffice'].mean()
        col_boxoffice_means[item] = mean_value
    target_encoding_list[col] = col_boxoffice_means

    df[f"{col}_target_transform"] = df[col].apply(lambda x : [col_boxoffice_means[s] for s in x])
    df[f"{col}_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : sorted(x,reverse=True))
    df[f"{col}_first_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : x[0] if len(x)>0 else 0)
    df[f"{col}_second_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : x[1] if len(x)>1 else 0)
    df[f"{col}_third_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : x[2] if len(x)>2 else 0)

json_data = json.dumps(target_encoding_list, ensure_ascii=False)
with open('target_encoding_list2.json', 'w', encoding='utf-8') as f:
    json.dump(target_encoding_list, f, indent=4, ensure_ascii=False)

In [6]:
df.columns

Index(['actors', 'critics_score', 'date', 'directors', 'editor', 'genre',
       'langage', 'length', 'nationality', 'viewers_score',
       'us_first_week_boxoffice', 'french_first_week_boxoffice', 'french_visa',
       'title', 'vo_title', 'url', 'genre_target_transform',
       'genre_first_target_transform', 'genre_second_target_transform',
       'genre_third_target_transform', 'langage_target_transform',
       'langage_first_target_transform', 'langage_second_target_transform',
       'langage_third_target_transform', 'nationality_target_transform',
       'nationality_first_target_transform',
       'nationality_second_target_transform',
       'nationality_third_target_transform', 'directors_target_transform',
       'directors_first_target_transform', 'directors_second_target_transform',
       'directors_third_target_transform', 'actors_target_transform',
       'actors_first_target_transform', 'actors_second_target_transform',
       'actors_third_target_transform'],
      

In [7]:
import pyarrow as pa
import pyarrow.parquet as pq
table = pa.Table.from_pandas(df)
pq.write_table(table, "targuet_transform2.parquet")

In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Preparation données

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GroupKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import lightgbm as lgbm
import optuna
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

df = df.drop(['critics_score','viewers_score','us_first_week_boxoffice','french_visa','title', 'vo_title', 'url'],axis=1)

# Extraction de caractéristiques à partir des colonnes de type liste
for col in ['genre', 'langage', 'nationality', 'actors']:
    df[col] = df[col].apply(lambda x : x if type(x) == np.ndarray else ["no value"])
# Nationalité du film
df['is_french_prod'] = df['nationality'].apply(lambda x: 1 if 'France' in x else 0)
df['is_usa_prod'] = df['nationality'].apply(lambda x: 1 if 'U.S.A.' in x or 'USA' in x or 'United States' in x else 0)
df['is_uk_prod'] = df['nationality'].apply(lambda x: 1 if 'Grande-Bretagne' in x or 'UK' in x or 'United Kingdom' in x else 0)
df['is_japan_prod'] = df['nationality'].apply(lambda x: 1 if 'Japon' in x or 'Japan' in x else 0)
df['is_italy_prod'] = df['nationality'].apply(lambda x: 1 if 'Italie' in x or 'Italy' in x else 0)
df['is_spain_prod'] = df['nationality'].apply(lambda x: 1 if 'Espagne' in x or 'Spain' in x else 0)
df['is_germany_prod'] = df['nationality'].apply(lambda x: 1 if 'Allemagne' in x or 'Germany' in x else 0)
df['nationality_count'] = df['nationality'].apply(len)  # Nombre de pays impliqués dans la production

# Langue du film
df['is_french_lang'] = df['langage'].apply(lambda x: 1 if 'Français' in x else 0)
df['is_english_lang'] = df['langage'].apply(lambda x: 1 if 'Anglais' in x else 0)
df['language_count'] = df['langage'].apply(len)  # Nombre de langues dans le film

# Caractéristiques temporelles
df['date'] = pd.to_datetime(df['date'])
df['release_year'] = df['date'].dt.year
df['release_month'] = df['date'].dt.month
df['release_day'] = df['date'].dt.day
df['release_day_of_week'] = df['date'].dt.dayofweek  # 0=lundi, 6=dimanche
df['is_summer_release'] = df['release_month'].isin([6, 7, 8]).astype(int)
df['is_winter_release'] = df['release_month'].isin([11, 12, 1]).astype(int)
df['is_holiday_release'] = ((df['release_month'] == 12) & (df['release_day'] >= 15)).astype(int)

# Nombre de réalisateurs
df['director_count'] = df['directors'].apply(len)

# Caractéristiques des acteurs
df['actor_count'] = df['actors'].apply(len)

# Fusion avec les données d'affluence mensuelle
def put_national_affluence(x, filled = False):
    date = pd.to_datetime(f"{x.year}-{x.month:02d}-01")
    if filled:
        return df_afluence_filled.loc[date, 'box_office'] if date in df_afluence_filled.index else None
    else:
        return df_afluence.loc[date, 'box_office'] if date in df_afluence.index else None
df['national_affluence'] = df['date'].apply(lambda x : put_national_affluence(x,True))

# Essayons d'intégrer les informations sur les acteurs
# Créer un dictionnaire pour chaque métrique d'acteur
actor_stats = {}
for index, row in df_actors.iterrows():
    name = index
    actor_stats[name] = {
        'amount_of_film': row['amount_of_film_played_in'],
        'avg_boxoffice': row['boxoffice_average'],
        'total_boxoffice': row['boxoffice_total'],
        'rank': row['rank']
    }

# Fonction pour extraire les statistiques des acteurs
def get_actor_features(actors_list):
    # Initialiser les variables
    count = 0
    total_avg_boxoffice = 0
    max_avg_boxoffice = 0
    avg_rank = 0
    found_actors = 0
    
    # Nettoyer les noms d'acteurs pour correspondre au format du dictionnaire
    cleaned_actors = [a.lower().replace(" ", "") for a in actors_list]
    
    for actor in cleaned_actors:
        if actor in actor_stats:
            found_actors += 1
            total_avg_boxoffice += actor_stats[actor]['avg_boxoffice']
            max_avg_boxoffice = max(max_avg_boxoffice, actor_stats[actor]['avg_boxoffice'])
            avg_rank += actor_stats[actor]['rank']
    
    # Éviter la division par zéro
    if found_actors > 0:
        return found_actors, total_avg_boxoffice / found_actors, max_avg_boxoffice, avg_rank / found_actors
    else:
        return 0, 0, 0, 0

# Appliquer la fonction à chaque ligne
actor_features = df['actors'].apply(get_actor_features)
df['found_actors'] = actor_features.apply(lambda x: x[0])
df['avg_actor_boxoffice'] = actor_features.apply(lambda x: x[1])
df['max_actor_boxoffice'] = actor_features.apply(lambda x: x[2])
df['avg_actor_rank'] = actor_features.apply(lambda x: x[3])

# Calculer le log de la target pour une distribution plus normale
df['log_french_boxoffice'] = np.log1p(df['french_first_week_boxoffice'])

def create_target_encoded_features(df, column, target_col='french_first_week_boxoffice'):
    # Vérifier le type
    df[column] = df[column].apply(lambda x: x if isinstance(x, (list, np.ndarray)) else ["no value"])
    
    # Créer le dictionnaire de mapping une seule fois
    unique_items = set()
    for item_list in df[column]:
        unique_items.update(item_list)
    
    # Calculer les moyennes pour chaque élément
    item_means = {}
    for item in unique_items:
        mask = df[column].apply(lambda x: item in x)
        mean_value = df.loc[mask, target_col].mean()
        item_means[item] = mean_value
    
    # Appliquer la transformation
    df[f"{column}_target_values"] = df[column].apply(lambda x: [item_means.get(s, 0) for s in x])
    sorted_values = df[f"{column}_target_values"].apply(lambda x: sorted(x, reverse=True))
    
    # Extraire les 3 premières valeurs (plus efficace)
    max_items = 3 
    for i in range(max_items):
        df[f"{column}_target_{i+1}"] = sorted_values.apply(lambda x: x[i] if len(x) > i else 0)
    
    # Ajouter la moyenne des valeurs comme feature
    df[f"{column}_target_mean"] = df[f"{column}_target_values"].apply(lambda x: sum(x)/len(x) if len(x) > 0 else 0)
    
    # Nettoyage des variables intermédiaires
    df.drop(f"{column}_target_values", axis=1, inplace=True)
    
    return df

for col in ['genre', 'langage', 'nationality', 'actors']:
    print(f"traintement de la col {col}")
    df = create_target_encoded_features(df, col, 'french_first_week_boxoffice')

# Détection et traitement des valeurs manquantes
print("Valeurs manquantes avant traitement:", df.isnull().sum())

print("Shape après prétraitement:", df.shape)

Valeurs manquantes avant traitement: actors                          0
date                            0
directors                       0
editor                          0
genre                           0
langage                         0
length                         12
nationality                     0
french_first_week_boxoffice     0
is_french_prod                  0
is_usa_prod                     0
is_uk_prod                      0
is_japan_prod                   0
is_italy_prod                   0
is_spain_prod                   0
is_germany_prod                 0
nationality_count               0
is_french_lang                  0
is_english_lang                 0
language_count                  0
release_year                    0
release_month                   0
release_day                     0
release_day_of_week             0
is_summer_release               0
is_winter_release               0
is_holiday_release              0
director_count                  0
actor_count

In [7]:
pd.set_option('display.max_columns', None)
df.head()

,actors,date,directors,editor,genre,langage,length,nationality,french_first_week_boxoffice,is_french_prod,is_usa_prod,is_uk_prod,is_japan_prod,is_italy_prod,is_spain_prod,is_germany_prod,nationality_count,is_french_lang,is_english_lang,language_count,release_year,release_month,release_day,release_day_of_week,is_summer_release,is_winter_release,is_holiday_release,director_count,actor_count,national_affluence,found_actors,avg_actor_boxoffice,max_actor_boxoffice,avg_actor_rank,log_french_boxoffice,genre_target_1,genre_target_2,genre_target_3,genre_target_mean,langage_target_1,langage_target_2,langage_target_3,langage_target_mean,nationality_target_1,nationality_target_2,nationality_target_3,nationality_target_mean,actors_target_1,actors_target_2,actors_target_3,actors_target_mean
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],110011.0,1,0,0,0,0,0,0,1,1,0,1,2023,9,27,2,0,0,0,1,16,8.521429,0,0.000000,0,0.0,11.608345,420527.528184,268761.801282,189607.906250,254995.144181,140468.087227,0.000000,0.000000,140468.087227,124385.111414,0.000000,0.000000,124385.111414,2.399840e+05,2.005255e+05,1.435692e+05,113237.455821
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],2905.0,1,0,0,0,0,0,0,1,1,1,4,2020,8,26,2,1,0,0,1,11,6.671429,3,826035.666667,959141,142.0,7.974533,381754.506923,141083.341007,0.000000,261418.923965,674122.505644,293844.488927,140604.275081,312259.839220,124385.111414,0.000000,0.000000,124385.111414,5.311230e+05,4.656778e+05,4.215080e+05,239902.252420
2,[Thom Hoffman],2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",39119.0,1,0,0,0,0,0,1,4,0,0,1,2024,8,7,2,1,0,0,2,1,14.505000,0,0.000000,0,0.0,10.574389,729750.547043,614841.678750,537705.826464,560014.722798,34498.542857,0.000000,0.000000,34498.542857,363953.987569,124385.111414,44294.738739,142593.872166,8.362500e+04,0.000000e+00,0.000000e+00,83625.000000
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],64039.0,1,0,0,0,0,0,0,1,1,0,1,2022,12,21,2,0,1,1,1,9,20.140000,2,469089.500000,489932,183.0,11.067263,141083.341007,0.000000,0.000000,141083.341007,140468.087227,0.000000,0.000000,140468.087227,124385.111414,0.000000,0.000000,124385.111414,2.117520e+05,1.555139e+05,1.499798e+05,111485.157904
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],113153.0,0,1,0,0,0,0,0,1,0,1,1,2020,1,29,2,0,1,0,1,26,14.573333,2,864868.000000,1069400,330.0,11.636505,211960.965418,141083.341007,0.000000,176522.153212,674122.505644,0.000000,0.000000,674122.505644,876430.876637,0.000000,0.000000,876430.876637,3.880101e+06,2.556313e+06,1.410191e+06,498109.732388


# train-test Split

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

def time_based_split(df, test_years=3, test_ratio=0.5):
    """
    Crée un découpage train/test basé sur le temps, avec stratification
    sur les colonnes 'french_prod' et 'french_first_week_boxoffice'.
    
    Args:
        df: DataFrame avec les données
        test_years: Nombre d'années récentes à considérer pour le découpage
        test_ratio: Proportion des films des années récentes à mettre dans le test set
    
    Returns:
        indices_train, indices_test
    """
    # S'assurer que la colonne date est bien en datetime
    df["date"] = pd.to_datetime(df["date"])
    
    # Déterminer la date limite pour les années récentes
    max_date = df["date"].max()
    cutoff_date = pd.Timestamp(year=max_date.year - test_years, month=1, day=1)
    
    # Masque pour les films récents
    recent_mask = df["date"] >= cutoff_date
    old_mask = ~recent_mask

    # Films anciens => 100% train
    old_indices = df[old_mask].index.tolist()
    
    # Films récents à splitter avec stratification
    recent_df = df[recent_mask].copy()
    
    # On crée une variable combinée pour la stratification
    recent_df["stratify_col"] = recent_df["is_french_prod"].astype(str) + "_" + pd.qcut(
        recent_df["french_first_week_boxoffice"], q=4, duplicates='drop'
    ).astype(str)

    # Appliquer le StratifiedShuffleSplit
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio, random_state=42)
    split = next(splitter.split(recent_df, recent_df["stratify_col"]))
    recent_train_idx, recent_test_idx = split

    # Récupérer les vrais indices du DataFrame original
    recent_indices = recent_df.index.tolist()
    test_indices = [recent_indices[i] for i in recent_test_idx]
    train_indices = old_indices + [recent_indices[i] for i in recent_train_idx]

    # Affichage
    print(f"Découpage temporel stratifié : {len(train_indices)} films pour l'entraînement, {len(test_indices)} films pour le test")
    print(f"Plage de dates d'entraînement : {df.loc[train_indices, 'date'].min()} à {df.loc[train_indices, 'date'].max()}")
    print(f"Plage de dates de test : {df.loc[test_indices, 'date'].min()} à {df.loc[test_indices, 'date'].max()}")

    return train_indices, test_indices

In [9]:
df.columns

Index(['actors', 'date', 'directors', 'editor', 'genre', 'langage', 'length',
       'nationality', 'french_first_week_boxoffice', 'is_french_prod',
       'is_usa_prod', 'is_uk_prod', 'is_japan_prod', 'is_italy_prod',
       'is_spain_prod', 'is_germany_prod', 'nationality_count',
       'is_french_lang', 'is_english_lang', 'language_count', 'release_year',
       'release_month', 'release_day', 'release_day_of_week',
       'is_summer_release', 'is_winter_release', 'is_holiday_release',
       'director_count', 'actor_count', 'national_affluence', 'found_actors',
       'avg_actor_boxoffice', 'max_actor_boxoffice', 'avg_actor_rank',
       'log_french_boxoffice', 'genre_target_1', 'genre_target_2',
       'genre_target_3', 'genre_target_mean', 'langage_target_1',
       'langage_target_2', 'langage_target_3', 'langage_target_mean',
       'nationality_target_1', 'nationality_target_2', 'nationality_target_3',
       'nationality_target_mean', 'actors_target_1', 'actors_target_2',
   

In [10]:
X=df.drop(['french_first_week_boxoffice',"log_french_boxoffice"], axis=1)
y=df['log_french_boxoffice']
train_indices, test_indices = time_based_split(df, test_years=3, test_ratio=0.5)
X_train, X_test = X.loc[train_indices], X.loc[test_indices]
y_train, y_test = y.loc[train_indices], y.loc[test_indices]

Découpage temporel stratifié : 8388 films pour l'entraînement, 763 films pour le test
Plage de dates d'entraînement : 1994-11-23 00:00:00 à 2025-05-28 00:00:00
Plage de dates de test : 2022-01-05 00:00:00 à 2025-04-09 00:00:00


# Création du modèle et de son preprocesseur

In [11]:
# Sélection des caractéristiques
# Exclure les colonnes qui ne sont pas utiles pour la modélisation
exclude_cols = ['actors', 'date', 'directors', 'genre', 'langage', 'nationality', 'french_first_week_boxoffice', 'log_french_boxoffice', 'target_quantile']
feature_cols = [col for col in X_train.columns if col not in exclude_cols]

# Séparation des types de features
categorical_features = ['editor', 'release_day_of_week']
numerical_features = [col for col in feature_cols if col not in categorical_features]

# Preprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Modèle LightGBM
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', lgbm.LGBMRegressor(
        objective='regression',
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

# Entraînement du modèle
model.fit(X_train[feature_cols], y_train)

# Évaluation initiale
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

y_pred = model.predict(X_test[feature_cols])
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")
print(f"MAE: {mae}")

# Exponentiel pour revenir à l'échelle originale
y_test_orig = np.expm1(y_test)
y_pred_orig = np.expm1(y_pred)
print(f"RMSE original: {np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))}")
print(f"MAE original: {mean_absolute_error(y_test_orig, y_pred_orig)}")

# Analyse des erreurs pour les top films
df_results = pd.DataFrame({
    'Actual': y_test_orig,
    'Predicted': y_pred_orig,
    'Error': y_test_orig - y_pred_orig,
    'AbsError': abs(y_test_orig - y_pred_orig),
    'RelError': abs((y_test_orig - y_pred_orig) / y_test_orig) * 100  # En pourcentage
})

# Top 10% des films selon le box-office réel
top_threshold = np.percentile(y_test_orig, 90)
top_films = df_results[df_results['Actual'] >= top_threshold]
print(f"RMSE pour les top 10% films: {np.sqrt(mean_squared_error(top_films['Actual'], top_films['Predicted']))}")
print(f"Erreur relative moyenne pour les top 10% films: {top_films['RelError'].mean()}%")

# Importance des caractéristiques
feature_importance = model.named_steps['regressor'].feature_importances_
feature_names = numerical_features + list(model.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print("Top 15 caractéristiques les plus importantes:")
print(importance_df.head(15))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001305 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3672
[LightGBM] [Info] Number of data points in the train set: 8388, number of used features: 126
[LightGBM] [Info] Start training from score 10.379418
RMSE: 1.1710946632283523
R²: 0.7696544943597394
MAE: 0.7999451065175224
RMSE original: 413534.3442538651
MAE original: 74427.1060351206
RMSE pour les top 10% films: 1290683.1479514004
Erreur relative moyenne pour les top 10% films: 44.12352235223864%
Top 15 caractéristiques les plus importantes:
                Feature  Importance
40   actors_target_mean        1515
37      actors_target_1        1024
39      actors_target_3         976
0                length         885
12         release_year         858
19          actor_count         837
38      actors_target_2         818
20   national_affluence         695
28    genre_target_mean         520


# Entrainement du modèle

In [12]:
# Définir une métrique personnalisée qui donne plus de poids aux films à succès
def custom_eval_metric(y_true, y_pred):
    # Convertir en valeurs originales
    y_true_orig = np.expm1(y_true)
    y_pred_orig = np.expm1(y_pred)
    
    # Identifier les top 10% films
    top_threshold = np.percentile(y_true_orig, 90)
    weights = np.where(y_true_orig >= top_threshold, 3.0, 1.0)  # 3x plus de poids aux top films
    
    # Calculer l'erreur pondérée
    errors = (y_true_orig - y_pred_orig) ** 2
    weighted_mse = np.sum(weights * errors) / np.sum(weights)
    return np.sqrt(weighted_mse)

# Cross-validation temporelle personnalisée
class TimeSeriesSplit:
    def __init__(self, n_splits=5):
        self.n_splits = n_splits
        
    def split(self, X, y=None, groups=None):
        X_dates = X['date'] if 'date' in X.columns else groups
        X_dates = pd.Series(X_dates)
        X_dates = pd.to_datetime(X_dates)
        X_sorted = X.sort_values('date') if 'date' in X.columns else X.iloc[X_dates.argsort()]
        
        n_samples = len(X_sorted)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = (i + 1) * k_fold_size if i < self.n_splits - 1 else n_samples
            mid = (start + stop) // 2
            yield indices[:mid], indices[mid:stop]

# Fonction objective pour Optuna
def objective(trial):
    # Hyperparamètres à optimiser
    params = {
        'objective': 'regression',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'random_state': 42
    }
    
    model = lgbm.LGBMRegressor(**params)
    
    # CV personnalisé
    cv = TimeSeriesSplit(n_splits=5)
    
    # Score avec validation croisée
    scores = []
    for train_idx, val_idx in cv.split(X_train, y_train, groups=X_train['date']):
        X_tr, X_val = X_train.iloc[train_idx][feature_cols], X_train.iloc[val_idx][feature_cols]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Prétraitement des données
        X_tr_transformed = preprocessor.fit_transform(X_tr)
        X_val_transformed = preprocessor.transform(X_val)
        
        # Entraînement
        model.fit(X_tr_transformed, y_tr)
        
        # Prédiction
        y_pred = model.predict(X_val_transformed)
        
        # Calcul du score personnalisé
        score = custom_eval_metric(y_val, y_pred)
        scores.append(score)
    
    return np.mean(scores)

# Exécution de l'optimisation Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Meilleurs hyperparamètres:", study.best_params)
print("Meilleur score:", study.best_value)

# Création du modèle final avec les meilleurs hyperparamètres
best_params = study.best_params
final_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', lgbm.LGBMRegressor(**best_params))
])

# Entraînement sur l'ensemble complet d'entraînement
final_model.fit(X_train[feature_cols], y_train)

# Évaluation finale
y_pred_final = final_model.predict(X_test[feature_cols])
y_test_orig_final = np.expm1(y_test)
y_pred_orig_final = np.expm1(y_pred_final)

print("Évaluation finale du modèle:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_final))}")
print(f"RMSE original: {np.sqrt(mean_squared_error(y_test_orig_final, y_pred_orig_final))}")
print(f"R²: {r2_score(y_test, y_pred_final)}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_final)}")

# Focus sur les top films
top_threshold_final = np.percentile(y_test_orig_final, 90)
top_films_mask = y_test_orig_final >= top_threshold_final
top_films_rmse = np.sqrt(mean_squared_error(
    y_test_orig_final[top_films_mask], 
    y_pred_orig_final[top_films_mask]
))
print(f"RMSE pour les top 10% films: {top_films_rmse}")

# Visualisation: prédiction vs réel
plt.figure(figsize=(12, 8))
plt.scatter(y_test_orig_final, y_pred_orig_final, alpha=0.5)
plt.plot([0, max(y_test_orig_final)], [0, max(y_test_orig_final)], 'r--')
plt.xlabel('Box Office Réel')
plt.ylabel('Box Office Prédit')
plt.title('Prédiction vs Réel (échelle originale)')
plt.xscale('log')
plt.yscale('log')
plt.grid(True)
plt.tight_layout()
plt.savefig('prediction_vs_real.png')

# Visualisation de l'importance des caractéristiques
feature_importance_final = final_model.named_steps['regressor'].feature_importances_
importance_df_final = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance_final
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(14, 10))
sns.barplot(x='Importance', y='Feature', data=importance_df_final.head(20))
plt.title('Top 20 caractéristiques importantes')
plt.tight_layout()
plt.savefig('feature_importance.png')

[I 2025-04-22 16:43:43,067] A new study created in memory with name: no-name-d8639b70-4497-42f1-94fe-1b306ece6243


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000314 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1908
[LightGBM] [Info] Number of data points in the train set: 838, number of used features: 29
[LightGBM] [Info] Start training from score 7.184861
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[W 2025-04-22 16:43:43,291] Trial 0 failed with parameters: {'n_estimators': 672, 'learning_rate': 0.0864586763284, 'num_leaves': 127, 'max_depth': 7, 'min_child_samples': 76, 'subsample': 0.8626921372038981, 'colsample_bytree': 0.6909179472702522, 'reg_alpha': 3.5397454412213305, 'reg_lambda': 3.958798346906429} because of the following error: ValueError('X has 411 features, but LGBMRegressor is expecting 270 features as input.').
Traceback (most recent call last):
  File "/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_39071/3481854319.py", line 70, in objective
    model.fit(X_tr_transformed, y_tr)
  File "/home/utilisateur/Documents/Simplon/new_is_always_better/.venv/lib/python3.12/site-packages/lightgbm/sklearn.py", line 1398, in fit
    super().fit(
  File "/home/utilisateur/Documents/Simplon/new

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

ValueError: X has 411 features, but LGBMRegressor is expecting 270 features as input.